In [8]:
from bs4 import BeautifulSoup
import requests, csv, re

In [154]:
main_source = r'https://www.imdb.com/chart/top/'
movie_list = r'https://www.imdb.com/search/title/?genres={}&sort=user_rating,desc&title_type=feature&num_votes=25000,&pf_rd_m=A2FGELUUNOQJNL&pf_rd_p=5aab685f-35eb-40f3-95f7-c53f09d542c3&pf_rd_r=Q3T2R6J68WYTY77VA6WR&pf_rd_s=right-6&pf_rd_t=15506&pf_rd_i=top&ref_=chttp_gnr_1'

In [155]:
def get_genre_tags():
    source_ = re.compile(r'/search/title\?genres=(.*)\&sort=user_rating,')
    source_html = requests.get(main_source)
    genre_tags = re.findall(source_, source_html.text)
    return genre_tags

In [159]:
def list_scraper(list_link, number=5, extract_metadata=True):
    all_movies = []
    movie_list_html = requests.get(list_link)
    for i in range(0, number):
        print('Taking care of {}/{}'.format(i, number))
        movie_dict = {
        'TITLE': None,
        'RUNTIME': None,
        'RATING': None
        }
        soup = BeautifulSoup(movie_list_html.text)
        movie_boxes = soup.findAll('div', class_='lister-item mode-advanced')
        movie_dict['TITLE'] = movie_boxes[i].find('h3', class_='lister-item-header').text.split('\n')[2]
        movie_dict['RUNTIME'] = movie_boxes[i].find('span', class_='runtime').text
        movie_dict['RATING'] = movie_boxes[i].find('div', class_='inline-block ratings-imdb-rating').text.strip()
        if extract_metadata:
            print("Getting metadata for ^")
            temp = movie_boxes[i].find('div', class_='lister-item-image float-left')
            movie_url = 'https://www.imdb.com' + temp.find('a').get('href')
            metadata_dict = movie_page(movie_url)
        movie_dict.update(metadata_dict)
        all_movies.append(movie_dict)
    return all_movies

In [160]:
def movie_page(movie_url):
    metadata_dict = {
        'DIRECTOR': None,
        'CAST': None,
        'STORYLINE': None
    }
    source = requests.get(movie_url)
    soup = BeautifulSoup(source.text)
    metadata_dict['DIRECTOR'] = soup.findAll('div', class_='credit_summary_item')[0].text.split('\n')[2]
    metadata_dict['STORYLINE'] = soup.findAll('div', class_='inline canwrap')[0].text.strip()
    all_cast = []
    cast_table = soup.find('table', class_='cast_list')
    cast_blocks = soup.findAll('tr', class_='odd')
    cast_blocks.append(soup.findAll('tr', class_='even'))
    for elem in cast_blocks:
        try:
            all_cast.append(elem.findAll('td')[1].text.strip())
        except Exception:
            continue
    metadata_dict['CAST'] = ','.join(all_cast)
    return metadata_dict

In [156]:
all_tags = get_genre_tags()

In [162]:
all_tags

['action',
 'adventure',
 'animation',
 'biography',
 'comedy',
 'crime',
 'drama',
 'family',
 'fantasy',
 'film_noir',
 'history',
 'horror',
 'music',
 'musical',
 'mystery',
 'romance',
 'sci_fi',
 'sport',
 'thriller',
 'war',
 'western']

In [164]:
final_movie_dict = []
for tag in all_tags[0:2]:
    print("Handling {}".format(movie_list.format(tag)))
    movie_db = list_scraper(movie_list.format(tag))
    final_movie_dict = final_movie_dict + movie_db

Handling https://www.imdb.com/search/title/?genres=action&sort=user_rating,desc&title_type=feature&num_votes=25000,&pf_rd_m=A2FGELUUNOQJNL&pf_rd_p=5aab685f-35eb-40f3-95f7-c53f09d542c3&pf_rd_r=Q3T2R6J68WYTY77VA6WR&pf_rd_s=right-6&pf_rd_t=15506&pf_rd_i=top&ref_=chttp_gnr_1
Taking care of 0/5
Getting metadata for ^
Taking care of 1/5
Getting metadata for ^
Taking care of 2/5
Getting metadata for ^
Taking care of 3/5
Getting metadata for ^
Taking care of 4/5
Getting metadata for ^
Handling https://www.imdb.com/search/title/?genres=adventure&sort=user_rating,desc&title_type=feature&num_votes=25000,&pf_rd_m=A2FGELUUNOQJNL&pf_rd_p=5aab685f-35eb-40f3-95f7-c53f09d542c3&pf_rd_r=Q3T2R6J68WYTY77VA6WR&pf_rd_s=right-6&pf_rd_t=15506&pf_rd_i=top&ref_=chttp_gnr_1
Taking care of 0/5
Getting metadata for ^
Taking care of 1/5
Getting metadata for ^
Taking care of 2/5
Getting metadata for ^
Taking care of 3/5
Getting metadata for ^
Taking care of 4/5
Getting metadata for ^


In [168]:
def to_csv(list_dict):
    keys = list_dict[0].keys()
    with open('people.csv', 'w') as output_file:
        dict_writer = csv.DictWriter(output_file, keys)
        dict_writer.writeheader()
        dict_writer.writerows(list_dict)

In [169]:
to_csv(final_movie_dict)